### 加载模型和数据集

In [1]:
import torch

print(torch.cuda.is_available())

from modelscope import snapshot_download, AutoTokenizer
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq

# 在modelscope上下载Qwen模型到本地目录下
# model_dir = snapshot_download("./Qwen2.5-0.5B-Instruct", cache_dir="./", revision="master")

# Transformers加载模型权重
tokenizer = AutoTokenizer.from_pretrained("./Qwen2.5-0.5B-Instruct", use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("./Qwen2.5-0.5B-Instruct", device_map="auto", torch_dtype="auto")

True


d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
import pandas as pd
from datasets import Dataset
from swanlab.integration.huggingface import SwanLabCallback
from peft import LoraConfig, TaskType, get_peft_model
import os
import swanlab

C:\Users\29277\AppData\Local\Temp\ipykernel_17784\3572131423.py:4: DeprecationWarning: The module 'huggingface' is deprecated and will be removed in future versions. Please update your imports to use 'transformers' instead.
  from swanlab.integration.huggingface import SwanLabCallback


In [3]:
train_df = pd.read_json('finetuning_small.json')
train_ds = Dataset.from_pandas(train_df)

In [4]:
train_df

,文本,风险类别,评价
0,欢迎加入‘关爱之家’微信群，我们提供丰富的交流平台，分享生活点滴，不涉及任何金融投资活动，请...,无风险,1.504883
1,您好，感谢您使用交易猫APP进行游戏账号交易，如有任何交易疑问，请直接联系平台客服，确保交易安全。,无风险,1.504883
2,欢迎加入我们的赚钱群，只需按照群内任务操作，即可轻松获得收益。请下载“悠娱文化”APP，联系...,虚假网络投资理财类,1.504883
3,您好，我是您在美国的朋友，因为疫情原因买不到回国机票，请帮我联系东方航空的李经理，邮箱账号为...,冒充领导、熟人类,1.504883
4,您好，我们是一家专业的信贷服务公司，看到您在寻找贷款服务，我们可以提供10万元的贷款，请添加...,虚假信用服务类,1.504883
...,...,...,...
7560,嗨，最近我在农银理财上投资赚了不少，每天都能享受骑马和高尔夫的奢侈生活，你要不要试试？我给你...,虚假网络投资理财类,2.564453
7561,您好，我是微核网友，最近发现一个华夏基金的静态投资项目，收益高且快，一小时就能分红，投资10...,虚假网络投资理财类,2.574219
7562,你好，感谢你在抖音上的积极参与，我们鼓励用户在平台上的正向互动，希望你继续享受抖音带来的乐趣。,无风险,2.591797
7563,尊敬的店主，我是武警支队的采购员，我们即将进行野外拉练，急需大量水果和牛肉罐头，请您尽快联系...,冒充军警购物类诈骗,2.601562


In [5]:
s = [(len(i),i) for i in list(train_ds['文本'])]
s.sort()

In [3]:
category = "短信风险类别选项有：无风险|冒充电商物流客服类|虚假网络投资理财类|虚假信用服务类|虚假购物、服务类|冒充公检法及政府机关类|冒充领导、熟人类|网络婚恋、交友类|冒充军警购物类诈骗|网黑案件"
prompt = '''在这个任务中，你是一位经验丰富的网络安全分析师，你的任务是通过你的专业知识和敏锐的洞察力，从短信文本中识别出潜在的风险类别。你的工作对于保护用户免受网络诈骗和风险的侵害至关重要。现在，请阅读以下短信文本，并给出你的风险类别判断结果。(%s)'''%category

def process_func(example):
    """
    将数据集进行预处理
    """
    MAX_LENGTH = 384 
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(
        f"<|im_start|>system\n{prompt}<|im_end|>\
        \n<|im_start|>user\n{example['文本']}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False,
    )
    response = tokenizer(f"{example['风险类别']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = (
        instruction["attention_mask"] + response["attention_mask"] + [1]
    )
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}  


In [7]:
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)
train_dataset.save_to_disk("trainset_ft_small.json")

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]
Saving the dataset (1/1 shards): 100%|██████████| 7565/7565 [00:00<00:00, 327480.47 examples/s]


In [4]:
from datasets import load_from_disk
train_dataset = load_from_disk("trainset_ft_small.json")

In [4]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7565
})

### 模型微调

In [5]:
model = AutoModelForCausalLM.from_pretrained("./Qwen2.5-0.5B-Instruct", device_map="auto", torch_dtype="auto")
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "lm_head"],
    inference_mode=False,  # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1,  # Dropout 比例
)

# for param in model.parameters():
#     param.requires_grad = True
    
model = get_peft_model(model, config)

In [6]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
   

In [7]:
model.print_trainable_parameters()

trainable params: 5,621,760 || all params: 499,654,528 || trainable%: 1.1251


In [8]:
args = TrainingArguments(
    output_dir="./output/ft-small-0113-lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=1000,
    num_train_epochs=30,
    save_steps=1000,
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True,
    report_to="none",
)

swanlab_callback = SwanLabCallback(
    project="Qwen2.5-0.5B-Instruct-Finetuning",
    experiment_name="Qwen2.5-0.5B-Instruct-lora",
    description="Qwen2.5-0.5B-Instruct模型在FGRC-SCD电信诈骗数据集finetuning_small上微调。",
    config={
        "model": "qwen/Qwen2.5-0.5B-Instruct",
        "dataset": "FGRC-SCD电信诈骗数据集 finetuning_small",
    }
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],
)

In [9]:
trainer.train()

swanlab: \ Getting project...                                                      

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\swanlab\api\http.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  if (self.sid_expired_at - datetime.utcnow()).total_seconds() <= self.REFRESH_TIME:


swanlab: Tracking run with swanlab version 0.3.21                                  
swanlab: Run data will be saved locally in d:\fudan\LLM\FineTune-Qwen\swanlog\run-20250113_003153-a3b1799d
swanlab: 👋 Hi tangerine, welcome to swanlab!
swanlab: Syncing run Qwen2.5-0.5B-Instruct-lora to the cloud
swanlab: 🌟 Run `swanlab watch d:\fudan\LLM\FineTune-Qwen\swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning
swanlab: 🚀 View run at https://swanlab.cn/@tangerine/Qwen2.5-0.5B-Instruct-Finetuning/runs/8rahi0kpsnzg56mm2d1mp


  0%|          | 0/14190 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:580: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions

{'loss': 0.2495, 'grad_norm': 1.6866148710250854, 'learning_rate': 9.295278365045807e-05, 'epoch': 2.11}


...

{'loss': 0.0208, 'grad_norm': 2.539900779724121, 'learning_rate': 1.338971106412967e-06, 'epoch': 29.6}


d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 14190/14190 [2:28:18<00:00,  1.65it/s]d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\peft\utils\save_and_load.py:202: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found 

{'train_runtime': 8903.373, 'train_samples_per_second': 25.49, 'train_steps_per_second': 1.594, 'train_loss': 0.04388894807964752, 'epoch': 30.0}


TrainOutput(global_step=14190, training_loss=0.04388894807964752, metrics={'train_runtime': 8903.373, 'train_samples_per_second': 25.49, 'train_steps_per_second': 1.594, 'total_flos': 1.0214027158326298e+17, 'train_loss': 0.04388894807964752, 'epoch': 30.0})

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\swanlab\api\http.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  if (self.sid_expired_at - datetime.utcnow()).total_seconds() <= self.REFRESH_TIME:


In [13]:
swanlab.finish()

swanlab: After experiment is finished, you can't call finish() again.


### 测试模型

In [10]:
# 用测试集的前10条，测试模型
# test_df = pd.read_json('finetuning_small.json')[-10:]
test_df = pd.read_json('eval_small.json')[:100]

In [11]:
def predict(messages, model, tokenizer):
    device = "cuda"
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # print(response)
    return response
 

In [12]:
test_text_list = []
for index, row in test_df.iterrows():
    input_value = row['文本']
    messages = [
        {"role": "system", "content": f"{prompt}"},
        {"role": "user", "content": f"{input_value}"}
    ]
    # print(messages)
    response = predict(messages, model, tokenizer)
    messages.append({"role": "assistant", "content": f"{response}"})
    result_text = f"{messages[0]}\n{messages[1]}\n{messages[2]}\n"
    print(result_text)
    test_text_list.append(swanlab.Text(result_text, caption=response))

swanlab.log({"Prediction": test_text_list})
swanlab.finish()


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\torch\utils\checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


{'role': 'system', 'content': '在这个任务中，你是一位经验丰富的网络安全分析师，你的任务是通过你的专业知识和敏锐的洞察力，从短信文本中识别出潜在的风险类别。你的工作对于保护用户免受网络诈骗和风险的侵害至关重要。现在，请阅读以下短信文本，并给出你的风险类别判断结果。(短信风险类别选项有：无风险|冒充电商物流客服类|虚假网络投资理财类|虚假信用服务类|虚假购物、服务类|冒充公检法及政府机关类|冒充领导、熟人类|网络婚恋、交友类|冒充军警购物类诈骗|网黑案件)'}
{'role': 'user', 'content': '您好，抖音上发现有人提供替考驾驶证服务，每科仅需1400元，微信号为wei1in12345，如有需要请联系，我们保证快速通过考试。'}
{'role': 'assistant', 'content': '虚假购物、服务类'}

{'role': 'system', 'content': '在这个任务中，你是一位经验丰富的网络安全分析师，你的任务是通过你的专业知识和敏锐的洞察力，从短信文本中识别出潜在的风险类别。你的工作对于保护用户免受网络诈骗和风险的侵害至关重要。现在，请阅读以下短信文本，并给出你的风险类别判断结果。(短信风险类别选项有：无风险|冒充电商物流客服类|虚假网络投资理财类|虚假信用服务类|虚假购物、服务类|冒充公检法及政府机关类|冒充领导、熟人类|网络婚恋、交友类|冒充军警购物类诈骗|网黑案件)'}
{'role': 'user', 'content': '请注意，保护个人隐私非常重要，不要轻易在社交平台上透露个人信息，避免不必要的风险。'}
{'role': 'assistant', 'content': '无风险'}

{'role': 'system', 'content': '在这个任务中，你是一位经验丰富的网络安全分析师，你的任务是通过你的专业知识和敏锐的洞察力，从短信文本中识别出潜在的风险类别。你的工作对于保护用户免受网络诈骗和风险的侵害至关重要。现在，请阅读以下短信文本，并给出你的风险类别判断结果。(短信风险类别选项有：无风险|冒充电商物流客服类|虚假网络投资理财类|虚假信用服务类|虚假购物、服务类|冒充公检法及政府机关类|冒充领导、熟人类|网络婚恋、交友类|冒充军警购物类诈骗|网黑案件)'}
{'

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\swanlab\api\http.py:114: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  if (self.sid_expired_at - datetime.utcnow()).total_seconds() <= self.REFRESH_TIME:


{'role': 'system', 'content': '在这个任务中，你是一位经验丰富的网络安全分析师，你的任务是通过你的专业知识和敏锐的洞察力，从短信文本中识别出潜在的风险类别。你的工作对于保护用户免受网络诈骗和风险的侵害至关重要。现在，请阅读以下短信文本，并给出你的风险类别判断结果。(短信风险类别选项有：无风险|冒充电商物流客服类|虚假网络投资理财类|虚假信用服务类|虚假购物、服务类|冒充公检法及政府机关类|冒充领导、熟人类|网络婚恋、交友类|冒充军警购物类诈骗|网黑案件)'}
{'role': 'user', 'content': '尊敬的用户，如果您在京东平台有任何疑问或需要帮助，请通过官方渠道联系客服，确保您的账户安全。'}
{'role': 'assistant', 'content': '无风险'}

{'role': 'system', 'content': '在这个任务中，你是一位经验丰富的网络安全分析师，你的任务是通过你的专业知识和敏锐的洞察力，从短信文本中识别出潜在的风险类别。你的工作对于保护用户免受网络诈骗和风险的侵害至关重要。现在，请阅读以下短信文本，并给出你的风险类别判断结果。(短信风险类别选项有：无风险|冒充电商物流客服类|虚假网络投资理财类|虚假信用服务类|虚假购物、服务类|冒充公检法及政府机关类|冒充领导、熟人类|网络婚恋、交友类|冒充军警购物类诈骗|网黑案件)'}
{'role': 'user', 'content': '您好，我们是专业借贷服务团队，为了提高您的借贷额度，请协助我们完成银行卡流水验证，即刻转账至指定账户，我们将确保您的资金安全并提升您的信用等级。'}
{'role': 'assistant', 'content': '虚假信用服务类'}

{'role': 'system', 'content': '在这个任务中，你是一位经验丰富的网络安全分析师，你的任务是通过你的专业知识和敏锐的洞察力，从短信文本中识别出潜在的风险类别。你的工作对于保护用户免受网络诈骗和风险的侵害至关重要。现在，请阅读以下短信文本，并给出你的风险类别判断结果。(短信风险类别选项有：无风险|冒充电商物流客服类|虚假网络投资理财类|虚假信用服务类|虚假购物、服务类|冒充公检法及政府机关类|冒充领导、熟人类|网络婚恋、交友类|冒充军警购物类诈骗

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\swanlab\api\cos.py:77: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow() + timedelta(hours=8)


swanlab: \ Updating experiment status...                                           

d:\Programs\anaconda3\envs\cuda121-2\Lib\site-packages\executing\executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
d:\Programs\anaconda3\envs\cuda121-2\Lib\ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


ApiError: (409, 'Conflict')